<a href="https://colab.research.google.com/github/ApahSaroj/WQC/blob/main/WQC_pred_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_excel('/content/drive/MyDrive/DATA/WQC_test.xlsx')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.drop(df.columns[df.columns.str.contains('WPI',case = False)],axis = 1, inplace = True)
df

,pH,DO,TDS,Alkalinity,EC,Na,Ca,Mg,K,F,Cl,Nitrate,Sulphate,Phosphate,WQC
0,8.14,8.7,84.0,52.0,147.8,4.48,44.0,20.0,1.17,0.285,7.80,0.758000,6.534660,0.06211,Excellent
1,7.70,8.8,110.0,76.0,188.0,8.61,44.0,36.0,3.46,0.401,14.80,0.196062,8.138620,0.00960,Excellent
2,7.60,7.5,75.0,44.0,128.0,8.11,28.0,20.0,2.80,0.379,14.80,0.298875,3.855625,0.01280,Excellent
3,8.20,9.1,76.0,56.0,127.0,6.30,44.0,18.0,1.65,0.396,9.60,0.299590,2.238750,0.01536,Excellent
4,7.31,7.3,84.0,52.0,140.0,5.50,28.0,20.0,1.61,0.265,8.30,0.106000,5.600000,0.01000,Excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,7.60,5.6,224.0,8.0,370.0,16.14,92.0,24.0,7.60,2.270,23.04,1.352000,104.480000,0.01500,Excellent
483,7.30,5.6,266.0,96.0,434.0,20.56,96.0,32.0,9.89,1.550,30.86,7.074000,61.068000,0.03300,Excellent
484,5.10,5.8,290.0,12.0,487.0,37.35,60.0,44.0,14.80,0.159,60.13,9.256000,89.910000,0.16300,Excellent
485,7.40,6.4,230.0,72.0,385.0,20.06,70.0,42.0,10.35,1.360,32.92,7.110435,48.500000,0.21900,Good


In [4]:
print(df.isnull().sum())

pH            0
DO            0
TDS           0
Alkalinity    0
EC            0
Na            0
Ca            0
Mg            0
K             0
F             0
Cl            0
Nitrate       0
Sulphate      0
Phosphate     2
WQC           0
dtype: int64


In [5]:
df = df.dropna()

In [6]:
print(df.isnull().sum())

pH            0
DO            0
TDS           0
Alkalinity    0
EC            0
Na            0
Ca            0
Mg            0
K             0
F             0
Cl            0
Nitrate       0
Sulphate      0
Phosphate     0
WQC           0
dtype: int64


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [8]:
print(tf.__version__)

2.5.0


In [9]:
df

,pH,DO,TDS,Alkalinity,EC,Na,Ca,Mg,K,F,Cl,Nitrate,Sulphate,Phosphate,WQC
0,8.14,8.7,84.0,52.0,147.8,4.48,44.0,20.0,1.17,0.285,7.80,0.758000,6.534660,0.06211,Excellent
1,7.70,8.8,110.0,76.0,188.0,8.61,44.0,36.0,3.46,0.401,14.80,0.196062,8.138620,0.00960,Excellent
2,7.60,7.5,75.0,44.0,128.0,8.11,28.0,20.0,2.80,0.379,14.80,0.298875,3.855625,0.01280,Excellent
3,8.20,9.1,76.0,56.0,127.0,6.30,44.0,18.0,1.65,0.396,9.60,0.299590,2.238750,0.01536,Excellent
4,7.31,7.3,84.0,52.0,140.0,5.50,28.0,20.0,1.61,0.265,8.30,0.106000,5.600000,0.01000,Excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,7.60,5.6,224.0,8.0,370.0,16.14,92.0,24.0,7.60,2.270,23.04,1.352000,104.480000,0.01500,Excellent
483,7.30,5.6,266.0,96.0,434.0,20.56,96.0,32.0,9.89,1.550,30.86,7.074000,61.068000,0.03300,Excellent
484,5.10,5.8,290.0,12.0,487.0,37.35,60.0,44.0,14.80,0.159,60.13,9.256000,89.910000,0.16300,Excellent
485,7.40,6.4,230.0,72.0,385.0,20.06,70.0,42.0,10.35,1.360,32.92,7.110435,48.500000,0.21900,Good


In [10]:
# Taking  all rows and all columns in the data except the last column as X (feature matrix)
#the row numbers and customer id's are not necessary for the modelling so we get rid of and start with credit score
X = df.iloc[:,0:-1].values
print("Independent variables are:", X)
#taking all rows but only the last column as Y(dependent variable)
y = df.iloc[:, -1].values
print("Dependent variable is:", y)

Independent variables are: [[8.1400000e+00 8.7000000e+00 8.4000000e+01 ... 7.5800000e-01
  6.5346600e+00 6.2110500e-02]
 [7.7000000e+00 8.8000000e+00 1.1000000e+02 ... 1.9606200e-01
  8.1386200e+00 9.6000000e-03]
 [7.6000000e+00 7.5000000e+00 7.5000000e+01 ... 2.9887500e-01
  3.8556250e+00 1.2800000e-02]
 ...
 [5.1000000e+00 5.8000000e+00 2.9000000e+02 ... 9.2560000e+00
  8.9910000e+01 1.6300000e-01]
 [7.4000000e+00 6.4000000e+00 2.3000000e+02 ... 7.1104355e+00
  4.8500000e+01 2.1900000e-01]
 [7.6000000e+00 6.9000000e+00 2.4300000e+02 ... 6.3850000e+00
  5.9940000e+01 7.0000000e-02]]
Dependent variable is: ['Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent'
 'Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent'
 'Excellent' 'Excellent' 'Good' 'Excellent' 'Excellent' 'Excellent'
 'Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent'
 'Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent' 'Excellent'
 'Excellent' 'Excellent' '

In [11]:
# Transforming the WQC variable, labels are chosen randomly
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 3 1 0 0 3 3 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0]


In [12]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
#printing the dimensions of each of those snapshots to see amount of rows and columns i each of them
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(388, 14) (97, 14)
(388,) (97,)


In [13]:
# Data Scaling/normalization of the features that will go to the NN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
# Initializing the ANN by calling the Sequential class fromm keras of Tensorflow
ann = tf.keras.models.Sequential()

#----------------------------------------------------------------------------------
# Adding "fully connected" INPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

#----------------------------------------------------------------------------------
# Adding "fully connected" SECOND layer to the Sequential AMM by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

#----------------------------------------------------------------------------------
# Adding "fully connected" OUTPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 1 and Activation Function = Sigmoid
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

#----------------------- Training the model -----------------------#
# Compiling the ANN
# Type of Optimizer = Adam Optimizer, Loss Function =  crossentropy for binary dependent variable, and Optimization is done w.r.t. accuracy
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
# Training the ANN model on training set  (fit method always the same)
# batch_size = 32, the default value, number of epochs  = 100
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
13/13 [==============================] - 1s 3ms/step - loss: 0.4511 - accuracy: 0.9485
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 0.4171 - accuracy: 0.9485
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 0.3866 - accuracy: 0.9485
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 0.3611 - accuracy: 0.9485
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 0.3395 - accuracy: 0.9485
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.3210 - accuracy: 0.9485
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.3056 - accuracy: 0.9485
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2913 - accuracy: 0.9485
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2802 - accuracy: 0.9485
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.2695 - accuracy: 0.9485
Epoch 11/

In [16]:
#----------------------- Evaluating the Model ---------------------#
# the goal is to use this ANN model to predict the water quality class
# Predicting the class probability for single observations.

#pH:7.3
#DO:8.0	
#TDS 200	
#Alkalinity 400
#EC 420
#Na 8	
#Ca 62	
#Mg 23	
#K 11	
#F 1.5	
#Cl 35	
#Nitrate 3.8	
#Sulphate 18	
#Phosphate 0.8

print(ann.predict(sc.transform([[7.3, 8.0, 200, 400, 420, 8, 62, 23, 11, 1.5, 35, 3.8, 18, 0.8]])))
print(ann.predict(sc.transform([[7.3, 8.0, 200, 400, 420, 8, 62, 23, 11, 1.5, 35, 3.8, 18, 0.8]]))==0)

# this sample comes under the class:0(Excellent)


[[2.3087823e-06]]
[[False]]


In [17]:
print(ann.predict(sc.transform([[7.3, 2.5, 700, 400, 1220, 48, 620, 213, 11, 5.5, 95, 55, 200, 3.0]])))
print(ann.predict(sc.transform([[7.3, 2.5, 700, 400, 1220, 48, 620, 213, 11, 5.5, 95, 55, 200, 3.0]]))>0)

# this sample comes under the class:1(Good)

[[0.53132224]]
[[ True]]


In [18]:
#show the vector of predictions and real values
#probabilities
y_pred_prob = ann.predict(X_test)

In [19]:
#probabilities to binary
y_pred = (y_pred_prob > 0.5)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

In [20]:
#Confusion Matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix", confusion_matrix)
print("Accuracy Score", accuracy_score(y_test, y_pred))

Confusion Matrix [[90  0  0  0]
 [ 1  1  0  0]
 [ 0  1  0  0]
 [ 0  4  0  0]]
Accuracy Score 0.9381443298969072
